In [1]:
import copy
import time

def run_test(cases, fun, should_fail = False):
    print(f"Testing '{fun.__name__}'")
    for args, expected in cases:
        
        fun_args = copy.deepcopy(args)
        start = time.time()
        result = fun(*fun_args)
        end = time.time()
        time_str = '%5.2f' % ((end - start) * 10**6)
        
        args_str = ",".join([f"'{arg}'" for arg in args])
        if result == expected:
            print(f"\tOK {time_str}ns {args_str[:35] + (args_str[35:] and '..')}")
        elif should_fail:           
            assert expected == result, f"FAIL {args_str}. Expected: '{expected}'. Got: '{result}'"
        else:
            print(f"\tFAIL {args_str}. Expected: '{expected}'. Got: '{result}'")

In [2]:
class Node:
    def __init__(self, value, prev = None, next = None):
        self.value = value
        self.prev = prev
        self.next = next
        
    def __str__(self):
        return f"N({self.value})"
        
class LinkedList:
    def __init__(self):
        self.head = None
    
    def append(self, node):
        if self.head == None:
            self.head = node
            return self
        
        cur = self.head
        
        while cur.next != None:
            cur = cur.next
            
        cur.next = node
        node.prev = cur
        
        return self
    
    def append_many(self, nodes):
        for i in range(len(nodes)):
            if i != len(nodes) - 1:
                nodes[i].next = nodes[i + 1]
            if i != [0]:
                nodes[i].prev = nodes[i - 1]
        self.append(nodes[0])
        
        return self
        
    def clear(self):
        cur = self.head
        
        while cur != None:
            cur, cur.prev, cur.next = cur.next, None, None
    
    def __eq__(self, other):
        cur_self = self.head
        cur_other = other.head
        
        while cur_self != None and cur_other != None:
            if cur_self.value != cur_other.value:
                return False
            
            cur_self = cur_self.next
            cur_other = cur_other.next
            
        return cur_self == cur_other
        
    def __str__(self):
        nodes = []
        
        cur = self.head
        while cur != None:
            nodes.append(cur)
            cur = cur.next
            
        return "↔".join([str(n.value) for n in nodes])
    
def as_list(values):
    assert len(values) > 0
    return LinkedList().append_many([Node(v) for v in values])


# 2.1 Remove Dups

Write code to remove duplicates from an unsorted linked list.

*FOLLOW UP:* How would you solve this problem if a temporary buffer is not allowed?

In [3]:
def remove_dups_quad(list: LinkedList):
    outer = list.head
    while outer != None:
        inner = outer.next
        
        while inner != None:
            next = inner.next
            if outer.value == inner.value:
                if inner.prev != None:
                    inner.prev.next = inner.next
                if inner.next != None:
                    inner.next.prev = inner.prev
                inner.prev, inner.next = None, None
            inner = next
        outer = outer.next
    return list

def remove_dups_hash(list):
    cur = list.head
    values = set()
    while cur != None:
        next = cur.next

        if cur.value in values:
            cur.prev.next = next
            if next != None:
                next.prev = cur.prev
            cur.prev, cur.next = None, None
        else:
            values.add(cur.value)
        cur = next

    return list

In [4]:
test_cases = [
    ([as_list([1, 2, 3])], as_list([1, 2, 3])),
    ([as_list([12, 11, 13, 14, 11, 30])], as_list([12, 11, 13, 14, 30])),
    ([as_list([1, 1, 1])], as_list([1])),
    ([as_list([1])], as_list([1])),
    ([as_list([1] * 300)], as_list([1]))
]

run_test(test_cases, remove_dups_quad)
run_test(test_cases, remove_dups_hash)

Testing 'remove_dups_quad'
	OK  3.81ns '1↔2↔3'
	OK  6.44ns '12↔11↔13↔14↔11↔30'
	OK  3.58ns '1↔1↔1'
	OK  1.91ns '1'
	OK 262.74ns '1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔..
Testing 'remove_dups_hash'
	OK  6.44ns '1↔2↔3'
	OK  8.34ns '12↔11↔13↔14↔11↔30'
	OK  6.44ns '1↔1↔1'
	OK  4.05ns '1'
	OK 205.76ns '1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔1↔..


# 2.2 Return Kth to Last

Implement an algorithm to find the kth to last element of a singly linked list.

In [81]:
class Node:
    def __init__(self, value):
        self.next = None
        self.value = value
        
    def __str__(self):
        nodes = []
        cur = self
        
        while cur != None:
            nodes.append(cur)
            cur = cur.next
            
        return "→".join(str(n.value) for n in nodes)
    
    def __eq__(self, other):
        if other == None:
            return False

        cur_self = self
        cur_other = other
        while cur_self != None and cur_other != None:
            if cur_self.value != cur_other.value:
                return False
            cur_self = cur_self.next
            cur_other = cur_other.next
            
        return cur_self == cur_other
        
def as_single_list(values):
    assert len(values) > 0
    nodes = [Node(v) for v in values]
    
    for i in range(len(nodes) - 1):
        nodes[i].next = nodes[i + 1]
    
    return nodes[0]
        
def kth_last_two_pass(node, k):
    assert k > 0
    cur = node
    n = 0
    while cur != None:
        cur = cur.next
        n += 1
    
    if k > n:
        return node
    
    cur = node
    for i in range(n - k):
        cur = cur.next
    return cur

def kth_last_extra_mem(node, k):
    assert k > 0
    nodes = []
    cur = node
    while cur != None:
        nodes.append(cur)
        cur = cur.next
    
    n = len(nodes)
    if k > n:
        return node
    return nodes[n - k]

def kth_last_two_pointers(node, k):
    assert k > 0
    
    fast = node
    fast_ind = 0
    
    slow = node
    slow_ind = 0
    
    while fast != None:
        fast = fast.next
        fast_ind += 1
        
        if fast_ind - k > slow_ind:
            slow = slow.next
            
    return slow

def kth_last_recursive(node, k):
    assert k > 0
    
    def kth_inner(n, i):
        if i == 0:
            return n, 0
        
        if n.next == None:
            return n, i - 1
        
        res, j = kth_inner(n.next, i)
        
        if j == 0:
            return res, 0
        return n, j - 1
    
    return kth_inner(node, k)[0]

In [82]:
test_cases = [
    ([as_single_list([1, 2, 3, 4, 5]), 1], as_single_list([5])),
    ([as_single_list([1] * 200), 200], as_single_list([1] * 200)),
    ([as_single_list([1, 2, 3, 4, 5]), 7], as_single_list([1, 2, 3, 4, 5])),
]

funs = [kth_last_two_pass, kth_last_extra_mem, kth_last_two_pointers, kth_last_recursive]
for f in funs:
    run_test(test_cases, f)

Testing 'kth_last_two_pass'
	OK 13.11ns '1→2→3→4→5','1'
	OK 79.15ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→..
	OK 30.04ns '1→2→3→4→5','7'
Testing 'kth_last_extra_mem'
	OK  5.01ns '1→2→3→4→5','1'
	OK 80.82ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→..
	OK 46.73ns '1→2→3→4→5','7'
Testing 'kth_last_two_pointers'
	OK  5.01ns '1→2→3→4→5','1'
	OK 73.67ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→..
	OK 23.60ns '1→2→3→4→5','7'
Testing 'kth_last_recursive'
	OK  5.01ns '1→2→3→4→5','1'
	OK 204.32ns '1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→1→..
	OK 107.77ns '1→2→3→4→5','7'
